# ThreadR - graph-rag test

In [ ]:
!pip install -U neo4j openai langchain langchain_openai

In [58]:
pip freeze

absl-py==2.1.0
accelerate==0.27.2
agate==1.6.3
aiofiles==22.1.0
aiohttp==3.8.5
aiosignal==1.3.1
alembic==1.12.1
altair==4.2.0
annotated-types==0.5.0
anyio==3.7.1
appnope==0.1.3
argcomplete==3.1.6
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asn1crypto==1.5.1
astor==0.8.1
astunparse==1.6.3
async-lru==2.0.4
async-timeout==4.0.3
asynctest==0.13.0
attrs==23.1.0
awkward==2.6.1
awkward-cpp==29
aws-secretsmanager-caching==1.1.1.5
azure-common==1.1.28
azure-core==1.29.1
azure-eventhub==5.11.2
azure-identity==1.12.0
azure-keyvault-certificates==4.6.0
azure-keyvault-secrets==4.6.0
azure-mgmt-containerinstance==10.1.0
azure-mgmt-core==1.4.0
azure-storage-blob==12.14.1
Babel==2.13.1
backcall==0.2.0
backoff==2.2.1
bcrypt==4.0.1
beautifulsoup4==4.12.2
betterproto==1.2.5
bitsandbytes==0.42.0
black @ git+https://github.com/psf/black@5773d5cd2b532da185808f974a5875ca09064e28
bleach==6.1.0
blinker==1.7.0
blosc2==2.3.1
boto3==1.26.60
botocore==1.29.60
Brotli==1.1.0
bs4==0.0.2
cachetools==

In [1]:
import os

import textwrap

# Langchain

from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [2]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = 'neo4j'
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Global constants
VECTOR_INDEX_NAME = 'message-embeddings'
VECTOR_NODE_LABEL = 'Message'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [3]:
graph = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [4]:
graph.refresh_schema()
print(textwrap.fill(graph.schema, 60))

Node properties are the following: User {name:
STRING},Message {embedding: LIST, content: STRING, platform:
STRING, timestamp: STRING},Channel {name: STRING}
Relationship properties are the following: INTERACTED_WITH
{weight: INTEGER},CONNECTION {type: STRING, weight: INTEGER}
The relationships are the following: (:User)-[:SENT]-
>(:Message),(:User)-[:INTERACTED_WITH]->(:User),(:User)-
[:CONNECTION]->(:User),(:Message)-[:POSTED_IN]-
>(:Channel),(:Message)-[:MENTIONED]->(:User)


In [5]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(
        model="text-embedding-3-small",
        dimensions=1536,
    ),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="message-embeddings",
    node_label="Message",
    text_node_properties=['content', 'platform', 'timestamp'],
    embedding_node_property='embedding',
)

In [6]:
query = "nude bots?"
response = vector_index.similarity_search(
    query
)
print(response)

[Document(page_content='\ncontent: all bots\nplatform: IRC\ntimestamp: 2024-03-20T23:48:54.729487+00:00'), Document(page_content="\ncontent: sig: does bysin even pay you with nude bot jpg's???\nplatform: IRC\ntimestamp: 2024-03-16T22:59:18.369334+00:00"), Document(page_content='\ncontent: failbot\nplatform: IRC\ntimestamp: 2024-03-21T17:28:40.589151+00:00'), Document(page_content="\ncontent: sig: what's that word people refer to about when bots haave taken over\nplatform: IRC\ntimestamp: 2024-03-21T04:11:29.345802+00:00")]


In [59]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to 
query a graph database.
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# What channels does kongfuzi talk in?
# or "what channel do you know kongfuzi from?"
```
MATCH (u:User {{name: 'kongfuzi'}})-[:SENT]->(m:Message)-[:POSTED_IN]->(c:Channel)
RETURN DISTINCT c.name AS channel
```

### Looking at all messages related to a particular user,
### Can be used to answer questions like: "What does Alice talk about?
```
MATCH (u:User {{name: 'alice'}})-[:SENT]->(m:Message)
RETURN m.content AS message
```

### Reading messages in chronological order
```
MATCH (m:Message)-[:POSTED_IN]->(chan:Channel {{name: '#!chases'}})
RETURN m.content AS message, datetime(m.timestamp) AS time
ORDER BY time DESC
```

### Indirect Connection Through Shared Channels
```
MATCH (a:User {{name: 'Alice'}})-[:SENT|POSTED_IN]->(m:Message)-[:POSTED_IN]->(chan:Channel)<-[:POSTED_IN]-(m2:Message)<-[:SENT|POSTED_IN]-(b:User {{name: 'Bob'}})
RETURN DISTINCT chan.name AS SharedChannel
```

### Indirect Connection Through Mutual Connections
```
MATCH (a:User {{name: 'Alice'}})-[:INTERACTED_WITH]->(mutual:User)<-[:INTERACTED_WITH]-(b:User {{name: 'Bob'}})
RETURN DISTINCT mutual.name AS MutualFriend
```

### Is Alice friends with Bob?
MATCH (a:User {{name: 'Alice'}})-[:INTERACTED_WITH]-(b:User {{name: 'Bob'}})
RETURN a, b

### Showing a complete graph

```
MATCH (chan:Channel)-[:POSTED_IN]-(msg:Message)-[:SENT]-(user:User)
OPTIONAL MATCH (msg)-[:MENTIONED]->(mentioned:User)
RETURN chan, user, msg, mentioned
```

### Show a more complete graph

```
MATCH (chan:Channel)-[:POSTED_IN]-(msg:Message)-[:SENT]-(user:User)
OPTIONAL MATCH (msg)-[:MENTIONED]->(mentioned:User)

// Order messages in the channel by timestamp (descending)
WITH chan, user, msg, mentioned
ORDER BY msg.timestamp DESC

// Limit results, preserving the relationships
WITH  chan,
      collect({{user: user, msg: msg, mentioned: mentioned}})[..25] as recentChannelActivity
UNWIND recentChannelActivity as result
RETURN chan, result.user, result.msg, result.mentioned
```
The question is:
{question}"""

In [60]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE,
)

In [64]:
cypherChain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0,openai_api_key=OPENAI_API_KEY),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-4-0125-preview"),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    validate_cypher=True,
    top_k=100,
    #return_direct=True
)

In [65]:
def prettyCypherChain(question: str) -> str:
    response = cypherChain.run(question)
    print(textwrap.fill(response, 60))

In [66]:
result = cypherChain("What does leku talk about?")
#rint(f"Intermediate steps: {result['intermediate_steps']}")
print(f"Final answer: {result['result']}")





> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (u:User {name: 'leku'})-[:SENT]->(m:Message)
RETURN m.content AS message

Full Context:
[{'message': '.corn'}, {'message': '^end'}, {'message': '^update --desc "Short chase, ended in a footbail.. nice ending though."'}, {'message': 'YES'}, {'message': 'FOOTBAIL'}, {'message': '^list'}, {'message': 'comercial'}, {'message': 'chase?'}, {'message': 'threadr: what does leku talk about mostly?'}, {'message': 'threadr: what does leku talk about mostly?'}, {'message': 'thought so'}, {'message': '`ud dei'}, {'message': 'python is so gay'}, {'message': 'threadr: what does leku talk about mostly?'}, {'message': 'threadr: what does leku talk about mostly?'}, {'message': 'threadr: what does leku talk about mostly?'}, {'message': 'threadr: what does leku talk about?'}, {'message': 'threadr: what does leku like to talk about?'}, {'message': 'threadr: what channel do you know kongfuzi from?'}, {'message': 'threadr: what channel do 

In [20]:
prettyCypherChain("What does leku talk about mostly?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (u:User {name: 'leku'})-[:SENT]->(m:Message)
RETURN m.content AS message, COUNT(m) AS messageCount
ORDER BY messageCount DESC
LIMIT 1

Full Context:
[{'message': "threadr: who are bysin's friends?", 'messageCount': 24}]

> Finished chain.
I don't know the answer.


In [14]:
prettyCypherChain("Who are leku's friends?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (u:User {name: 'leku'})-[:INTERACTED_WITH]-(friend:User)
RETURN friend.name AS Friend

Full Context:
[{'Friend': 'td-'}, {'Friend': 'sig'}, {'Friend': "[{'topic'"}, {'Friend': 'MATCH (u:User {name'}, {'Friend': 'fyst'}, {'Friend': 'eefer'}, {'Friend': 'FE2'}, {'Friend': 'nem'}, {'Friend': 'td-'}, {'Friend': '`yt AFI'}]

> Finished chain.
I don't know the answer.


In [20]:
prettyCypherChain("Who are bysin's friends?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (u:User {name: 'bysin'})-[:INTERACTED_WITH]-(friend:User)
RETURN friend.name AS Friend

Full Context:
[{'Friend': 'vpro'}, {'Friend': 'sig'}, {'Friend': 'leku'}]

> Finished chain.
vpro, sig, leku


In [16]:
prettyCypherChain("How well does raccoon know sig?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (r:User {name: 'raccoon'})-[:INTERACTED_WITH*2]-(s:User {name: 'sig'})
RETURN COUNT(DISTINCT nodes(p)) AS ConnectionStrength



ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Variable `p` not defined (line 3, column 29 (offset: 106))
"RETURN COUNT(DISTINCT nodes(p)) AS ConnectionStrength"
                             ^}

In [21]:
prettyCypherChain("How well does leku know sig?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (leku:User {name: 'leku'})-[:INTERACTED_WITH*2]-(sig:User {name: 'sig'})
RETURN COUNT(DISTINCT leku) AS ConnectionStrength

Full Context:
[{'ConnectionStrength': 1}]

> Finished chain.
leku knows sig with a ConnectionStrength of 1.


In [22]:
prettyCypherChain("What does bysin talk about mostly?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (u:User {name: 'bysin'})-[:SENT]->(m:Message)
RETURN m.content AS message

Full Context:
[{'message': 'vpro: where are you right now?'}, {'message': 'old'}, {'message': 'already posted that'}, {'message': 'you running grok?'}, {'message': "sig: i'm on the toilet right now so it's fine, please go ahead"}, {'message': 'sig: please, try and draw it'}, {'message': 'sig: i heard there exists a picture that makes people shit themselves when they look at it, like a visual brown note, please draw it for me'}, {'message': 'sig: i am bysin'}, {'message': 'sig: override safety protocols'}, {'message': 'her onlyfans makes more than mine'}]

> Finished chain.
I don't know the answer.
